# Resnet Finetuning on MSCOCO

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from utils.nn_visualization import init_embedding_projector, init_embedding_data
from utils.nn_visualization import get_sprite_img, get_label_class_names
from utils.nn_graph import simple_layer, variable_summaries
from utils.data import init_model_logging
from utils.data import Dataset

import numpy as np
import tensorflow as tf
from tensorflow.contrib.slim.nets import resnet_v1
from tensorflow.contrib import slim

## Load data

In [ ]:
class data:
    train = Dataset('/data/mscoco/train_data.hdf', '/data/mscoco/train_imgs/', one_hot=True, norm=False)
    validation = Dataset('/data/mscoco/valid_data.hdf', '/data/mscoco/valid_imgs/', one_hot=True, norm=False)
    
class_id2class_name_mapping = {
    0: 'cow',
    1: 'sheep',
    2: 'giraffe',
    3: 'horse',
    4: 'bird',
    5: 'cat',
    6: 'dog',
    7: 'elephant',
    8: 'bear'}

## Build Resnet Graph

In [ ]:
resnet_model_ckpt = '/data/checkpoints/resnet_v1_50.ckpt'
graph = tf.Graph()
with graph.as_default():
    with tf.variable_scope('resnet_inputs'):
        images = tf.placeholder(dtype=tf.float32, shape=[None, 224 * 224 * 3], name='images')
        labels = tf.placeholder(tf.float32, shape=(None, 9), name='labels')
        is_training = tf.placeholder(tf.bool, shape=(), name='training_flag')

    with tf.name_scope('image_reshape'):        
        images_reshaped = tf.reshape(images, [-1, 224, 224, 3])        

    with slim.arg_scope(resnet_v1.resnet_arg_scope()):
        resnet_tensors = resnet_v1.resnet_v1_50(images_reshaped,
                                                is_training=is_training,
                                                num_classes=None,
                                                global_pool=True,
                                                scope='resnet_v1_50')
    resnet_feature_vector, resnet_endpoints = resnet_tensors

    with tf.name_scope('resnet_feature_vector'):
        resnet_feature_vector = tf.reshape(resnet_feature_vector, shape=(-1, 2048))

    with tf.variable_scope('layer1'):
        prediction_raw = resnet_feature_vector
        prediction_raw = simple_layer('layer1', prediction_raw, shape=[2048, 1024], activation='relu')
        
    with tf.variable_scope('layer2'):
        prediction_raw = simple_layer('layer2', prediction_raw, shape=[1024, 9], activation='linear')

    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(prediction_raw)

    with tf.name_scope('loss'):
        cross_entropy_vector = tf.losses.softmax_cross_entropy(logits=prediction_raw, onehot_labels=labels)
        variable_summaries('loss_summary', cross_entropy_vector)
        loss = tf.reduce_mean(cross_entropy_vector)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)
        
    with tf.name_scope("embedding_visualization"):
        embedding = tf.Variable(tf.zeros([480, 2048]), name='valid_embedding')
        embedding_assignment = embedding.assign(resnet_feature_vector)       
    
    with tf.name_scope('training'):
        loss_to_optimize = loss
        
        # Select subsets of variables we want to train
        var_all = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
        var_resnet = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='resnet_v1_50')
        var_resnet_3_and_4 = list(filter(lambda var: 'block3' in var.name or 'block4' in var.name, var_resnet))
        var_top = list(set(var_all) - set(var_resnet))

        # Define gradinets with respect to loss
        gradients = tf.gradients(loss_to_optimize, var_resnet_3_and_4 + var_top)
        gradients_resnet = gradients[:len(var_resnet_3_and_4)]
        gradients_top = gradients[len(var_resnet_3_and_4):]

        # Setup 2 optimizers
        optimizer_resnet = tf.train.AdamOptimizer(0.0001)
        optimizer_top = tf.train.AdamOptimizer(0.001)

        # Substitute minimize method with gradients and apply_gradients
        train_resnet = optimizer_resnet.apply_gradients(zip(gradients_resnet, var_resnet_3_and_4))
        train_top = optimizer_top.apply_gradients(zip(gradients_top, var_top))
        
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_step = tf.group(train_resnet, train_top)
            #train_step = tf.train.AdamOptimizer(0.001).minimize(loss_to_optimize)
            
    init_resnet = slim.assign_from_checkpoint_fn(resnet_model_ckpt, slim.get_model_variables('resnet_v1_50'))
    initialize_vars = tf.group(
        tf.global_variables_initializer(),
        tf.local_variables_initializer())
    merge_summaries = tf.summary.merge_all()

## Init Model Logging

In [ ]:
base_dir = '/tensorboard_summaries/resnet/'
#exp_name = 'experiment_finetune_one_optimizer'
exp_name = 'experiment_finetune_two_optimizers'

logging_meta = init_model_logging(base_dir, exp_name, graph=graph, remove_existing=False)

## Load Pretrained Resnet

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']+'-1700'

validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    is_training: False}

session = tf.Session(graph=graph, config=config)
logging_meta['saver'].restore(session, model_path)
_prediction, = session.run([prediction], validation_feed_dict)

In [ ]:
session.close()

## Add Embedding Projection

In [ ]:
projection_image_shape = [80, 80, 3]
training_image_shape = [224, 224, 3]

In [ ]:
data.validation.image_shape = projection_image_shape
sprite_img = get_sprite_img(data.validation.images, projection_image_shape)
data.validation.image_shape = training_image_shape
sprite_img

In [ ]:
label_names = get_label_class_names(data.validation.labels, class_id2class_name_mapping)

In [ ]:
init_embedding_data(logging_meta['valid_writer_dir'], sprite_img, label_names)
init_embedding_projector(logging_meta['valid_writer'], embedding, projection_image_shape)

## Run Resnet

In [ ]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']

validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    is_training: False}

with tf.Session(graph=graph, config=config) as session:
    session.run(initialize_vars)
    init_resnet(session)
    for iteration in range(1901):
        ##################
        # Training phase #
        ##################
        _images, _labels = data.train.next_batch(30)
        feed_dict={images: _images, labels: _labels, is_training: True}
        _ = session.run([train_step], feed_dict)
        
        if iteration % 10 == 0:
            feed_dict={images: _images, labels: _labels, is_training: False}
            _summary,  _accuracy, _loss = session.run([merge_summaries, accuracy, loss], feed_dict)
            print("Iteration Train {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))
            logging_meta['train_writer'].add_summary(_summary, iteration)
        
        ####################
        # Validation phase #
        ####################    
        if iteration % 100 == 0:
            fetches = [embedding_assignment, merge_summaries, accuracy, loss]
            _, _summary, _accuracy, _loss = session.run(fetches, validation_feed_dict)
            logging_meta['valid_writer'].add_summary(_summary, iteration)
            logging_meta['saver'].save(session, model_path, iteration)
            print("= Valid Iteration {}: loss {}, accuracy {} =".format(iteration, _loss, _accuracy))
    
    _prediction, = session.run([prediction], validation_feed_dict)

## Results evaluation

In [ ]:
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)

In [ ]:
get_accuracy(df, use_top3=False)

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

In [ ]:
plot_coocurance_matrix(df, use_log=False)

In [ ]:
fp = get_false_positives(df, 'horse')

In [ ]:
plot_examples(fp, [224, 224 , 3])